![alt text](http://pandas.pydata.org/_static/pandas_logo.png)

<center><h1> PANDAS TABLE OF CONTENTS </h1></center>

## [Working with Text Data](http://pandas.pydata.org/pandas-docs/stable/text.html) 

- [Splitting-and-Replacing-Strings](#Splitting-and-Replacing-Strings)
- [Indexing with .str](#Indexing-with-.str)
- [Extracting Substrings](#Extracting-Substrings)
- [Testing for Strings that Match or Contain a Pattern](#Testing-for-Strings-that-Match-or-Contain-a-Pattern)
- [Creating Indicator Variables](#Creating-Indicator-Variables)
- [Method Summary](#Method-Summary)


In [1]:
import pandas as pd
import numpy as np

Series and Index are equipped with a set of string processing methods that make it easy to operate on each element of the array. Perhaps most importantly, these methods exclude missing/NA values automatically. These are accessed via the str attribute and generally have names matching the equivalent (scalar) built-in string methods:

In [2]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])

In [3]:
s

0       A
1       B
2       C
3    Aaba
4    Baca
5     NaN
6    CABA
7     dog
8     cat
dtype: object

In [4]:
s.str.lower()

0       a
1       b
2       c
3    aaba
4    baca
5     NaN
6    caba
7     dog
8     cat
dtype: object

In [5]:
s.str.upper()

0       A
1       B
2       C
3    AABA
4    BACA
5     NaN
6    CABA
7     DOG
8     CAT
dtype: object

In [6]:
s.str.len()

0    1.0
1    1.0
2    1.0
3    4.0
4    4.0
5    NaN
6    4.0
7    3.0
8    3.0
dtype: float64

In [7]:
idx = pd.Index([' jack', 'jill ', ' jesse ', 'frank'])

In [8]:
idx

Index([u' jack', u'jill ', u' jesse ', u'frank'], dtype='object')

In [9]:
idx.str.strip()

Index([u'jack', u'jill', u'jesse', u'frank'], dtype='object')

In [10]:
idx.str.lstrip()

Index([u'jack', u'jill ', u'jesse ', u'frank'], dtype='object')

In [11]:
idx.str.rstrip()

Index([u' jack', u'jill', u' jesse', u'frank'], dtype='object')

The string methods on Index are especially useful for cleaning up or transforming DataFrame columns. For instance, you may have columns with leading or trailing whitespace:

In [12]:
df = pd.DataFrame(np.random.randn(3, 2), columns=[' Column A ', ' Column B '],
                  index=range(3))

In [13]:
df

,Column A,Column B
0,-0.343147,-0.240406
1,-1.727499,-0.914029
2,-0.245533,-0.713243


Since df.columns is an Index object, we can use the .str accessor

In [14]:
df.columns.str.strip()

Index([u'Column A', u'Column B'], dtype='object')

In [15]:
df.columns.str.lower()

Index([u' column a ', u' column b '], dtype='object')

These string methods can then be used to clean up the columns as needed. Here we are removing leading and trailing whitespaces, lowercasing all names, and replacing any remaining whitespaces with underscores:

In [16]:
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

In [17]:
df

,column_a,column_b
0,-0.343147,-0.240406
1,-1.727499,-0.914029
2,-0.245533,-0.713243


**Note:** If you have a Series where lots of elements are repeated (i.e. the number of unique elements in the Series is a lot smaller than the length of the Series), it can be faster to convert the original Series to one of type category and then use .str.<method> or .dt.<property> on that. The performance difference comes from the fact that, for Series of type category, the string operations are done on the .categories and not on each element of the Series.
Please note that a Series of type category with string .categories has some limitations in comparison of Series of type string (e.g. you can’t add strings to each other: s + " " + s won’t work if s is a Series of type category). Also, .str methods which operate on elements of type list are not available on such a Series.

## Splitting and Replacing Strings

Methods like split return a Series of lists:

In [18]:
s2 = pd.Series(['a_b_c', 'c_d_e', np.nan, 'f_g_h'])

In [19]:
s2

0    a_b_c
1    c_d_e
2      NaN
3    f_g_h
dtype: object

In [20]:
s2.str.split('_')

0    [a, b, c]
1    [c, d, e]
2          NaN
3    [f, g, h]
dtype: object

Elements in the split lists can be accessed using get or [] notation:

In [21]:
s2.str.split('_').str.get(1)

0      b
1      d
2    NaN
3      g
dtype: object

In [22]:
s2.str.split('_').str[1]

0      b
1      d
2    NaN
3      g
dtype: object

Easy to expand this to return a DataFrame using expand.

In [23]:
s2.str.split('_', expand=True)

,0,1,2
0,a,b,c
1,c,d,e
2,NaN,None,None
3,f,g,h


It is also possible to limit the number of splits:

In [24]:
s2.str.split('_', expand=True, n=1)

,0,1
0,a,b_c
1,c,d_e
2,NaN,None
3,f,g_h


rsplit is similar to split except it works in the reverse direction, i.e., from the end of the string to the beginning of the string:

In [25]:
s2.str.rsplit('_', expand=True, n=1)

,0,1
0,a_b,c
1,c_d,e
2,NaN,None
3,f_g,h


Methods like replace and findall take [regular expressions](https://docs.python.org/2/library/re.html), too:

In [26]:
s3 = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca',
                '', np.nan, 'CABA', 'dog', 'cat'])

In [27]:
s3

0       A
1       B
2       C
3    Aaba
4    Baca
5        
6     NaN
7    CABA
8     dog
9     cat
dtype: object

In [28]:
 s3.str.replace('^.a|dog', 'XX-XX ', case=False)

0           A
1           B
2           C
3    XX-XX ba
4    XX-XX ca
5            
6         NaN
7    XX-XX BA
8      XX-XX 
9     XX-XX t
dtype: object

Some caution must be taken to keep regular expressions in mind! For example, the following code will cause trouble because of the regular expression meaning of $:

In [29]:
# Consider the following badly formatted financial data
dollars = pd.Series(['12', '-$10', '$10,000'])

In [30]:
dollars

0         12
1       -$10
2    $10,000
dtype: object

In [31]:
dollars.str.replace('$', '')

0        12
1       -10
2    10,000
dtype: object

In [32]:
# But this doesn't:
dollars.str.replace('-$', '-')

0         12
1       -$10
2    $10,000
dtype: object

In [33]:
# We need to escape the special character (for >1 len patterns)
dollars.str.replace(r'-\$', '-')

0         12
1        -10
2    $10,000
dtype: object

## Indexing with .str

You can use [] notation to directly index by position locations. If you index past the end of the string, the result will be a NaN.

In [34]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan,
               'CABA', 'dog', 'cat'])

In [35]:
s

0       A
1       B
2       C
3    Aaba
4    Baca
5     NaN
6    CABA
7     dog
8     cat
dtype: object

In [36]:
s.str[0:2]

0      A
1      B
2      C
3     Aa
4     Ba
5    NaN
6     CA
7     do
8     ca
dtype: object

In [37]:
s.str[1]

0    NaN
1    NaN
2    NaN
3      a
4      a
5    NaN
6      A
7      o
8      a
dtype: object

## Extracting Substrings

### Extract first match in each subject (extract)

New in version 0.13.0.

**Warning:** In version 0.18.0, extract gained the expand argument. When expand=False it returns a Series, Index, or DataFrame, depending on the subject and regular expression pattern (same behavior as pre-0.18.0). When expand=True it always returns a DataFrame, which is more consistent and less confusing from the perspective of a user.

The extract method accepts a [regular expression](https://docs.python.org/2/library/re.html) with at least one capture group.

Extracting a regular expression with more than one group returns a DataFrame with one column per group.

In [38]:
 pd.Series(['a1', 'b2', 'c3'])

0    a1
1    b2
2    c3
dtype: object

In [39]:
pd.Series(['a1', 'b2', 'c3']).str.extract('([ab])(\d)', expand=False)

,0,1
0,a,1
1,b,2
2,NaN,NaN


Elements that do not match return a row filled with NaN. Thus, a Series of messy strings can be “converted” into a like-indexed Series or DataFrame of cleaned-up or more useful strings, without necessitating get() to access tuples or re.match objects. The dtype of the result is always object, even if no match is found and the result only contains NaN.

Named groups like

In [40]:
pd.Series(['a1', 'b2', 'c3']).str.extract('(?P<letter>[ab])(?P<digit>\d)', expand=False)

,letter,digit
0,a,1
1,b,2
2,NaN,NaN


and optional groups like

In [41]:
pd.Series(['a1', 'b2', '3']).str.extract('([ab])?(\d)', expand=False)

,0,1
0,a,1
1,b,2
2,NaN,3


can also be used. Note that any capture group names in the regular expression will be used for column names; otherwise capture group numbers will be used.

Extracting a regular expression with one group returns a DataFrame with one column if expand=True.

In [42]:
pd.Series(['a1', 'b2', 'c3']).str.extract('[ab](\d)', expand=True)

,0
0,1
1,2
2,NaN


It returns a Series if expand=False.

In [43]:
pd.Series(['a1', 'b2', 'c3']).str.extract('[ab](\d)', expand=False)

0      1
1      2
2    NaN
dtype: object

Calling on an Index with a regex with exactly one capture group returns a DataFrame with one column if expand=True,

In [44]:
s = pd.Series(["a1", "b2", "c3"], ["A11", "B22", "C33"])

In [45]:
s

A11    a1
B22    b2
C33    c3
dtype: object

In [46]:
s.index.str.extract("(?P<letter>[a-zA-Z])", expand=True)

,letter
0,A
1,B
2,C


It returns an Index if expand=False.

In [47]:
s.index.str.extract("(?P<letter>[a-zA-Z])", expand=False)

Index([u'A', u'B', u'C'], dtype='object', name=u'letter')

Calling on an Index with a regex with more than one capture group returns a DataFrame if expand=True.

In [48]:
s.index.str.extract("(?P<letter>[a-zA-Z])([0-9]+)", expand=True)

,letter,1
0,A,11
1,B,22
2,C,33


It raises ValueError if expand=False.

    s.index.str.extract("(?P<letter>[a-zA-Z])([0-9]+)", expand=False)
    
    ValueError: only one regex group is supported with Index

The table below summarizes the behavior of extract(expand=False) (input subject in first column, number of groups in regex in first row)

<table border="1" class="docutils" align="left">
<colgroup>
<col width="28%" />
<col width="31%" />
<col width="41%" />
</colgroup>
<tbody valign="top">
<tr class="row-odd"><td>&nbsp;</td>
<td>1 group</td>
<td>&gt;1 group</td>
</tr>
<tr class="row-even"><td>Index</td>
<td>Index</td>
<td>ValueError</td>
</tr>
<tr class="row-odd"><td>Series</td>
<td>Series</td>
<td>DataFrame</td>
</tr>
</tbody>
</table>

### Extract all matches in each subject (extractall)

New in version 0.18.0.

Unlike extract (which returns only the first match),

In [49]:
s = pd.Series(["a1a2", "b1", "c1"], ["A", "B", "C"])

In [50]:
s

A    a1a2
B      b1
C      c1
dtype: object

In [51]:
two_groups = '(?P<letter>[a-z])(?P<digit>[0-9])'

In [52]:
s.str.extract(two_groups, expand=True)

,letter,digit
A,a,1
B,b,1
C,c,1


the extractall method returns every match. The result of extractall is always a DataFrame with a MultiIndex on its rows. The last level of the MultiIndex is named match and indicates the order in the subject.

In [53]:
 s.str.extractall(two_groups)

letter digit
  match             
A 0          a     1
  1          a     2
B 0          b     1
C 0          c     1

When each subject string in the Series has exactly one match,

In [54]:
s = pd.Series(['a3', 'b3', 'c2'])

In [55]:
s

0    a3
1    b3
2    c2
dtype: object

then extractall(pat).xs(0, level='match') gives the same result as extract(pat).

In [56]:
extract_result = s.str.extract(two_groups, expand=True)

In [57]:
extract_result

,letter,digit
0,a,3
1,b,3
2,c,2


In [58]:
extractall_result = s.str.extractall(two_groups)

In [59]:
extractall_result

,,letter,digit
,match,,
0,0,a,3
1,0,b,3
2,0,c,2


In [60]:
extractall_result.xs(0, level="match")

,letter,digit
0,a,3
1,b,3
2,c,2


## Testing for Strings that Match or Contain a Pattern

You can check whether elements contain a pattern:

In [61]:
pattern = r'[a-z][0-9]'

In [62]:
pd.Series(['1', '2', '3a', '3b', '03c']).str.contains(pattern)

0    False
1    False
2    False
3    False
4    False
dtype: bool

or match a pattern:

In [63]:
pd.Series(['1', '2', '3a', '3b', '03c']).str.match(pattern)

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [64]:
pd.Series(['1', '2', '3a', '3b', '03c']).str.match(pattern, as_indexer=True)

0    False
1    False
2    False
3    False
4    False
dtype: bool

The distinction between match and contains is strictness: match relies on strict re.match, while contains relies on re.search.

**Warning:** In previous versions, match was for extracting groups, returning a not-so-convenient Series of tuples. The new method extract (described in the previous section) is now preferred.

This old, deprecated behavior of match is still the default. As demonstrated above, use the new behavior by setting as_indexer=True. In this mode, match is analogous to contains, returning a boolean Series. The new behavior will become the default behavior in a future release.

Methods like match, contains, startswith, and endswith take
an extra na argument so missing values can be considered True or False:

In [65]:
s4 = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])

In [66]:
s4.str.contains('A')

0     True
1    False
2    False
3     True
4    False
5      NaN
6     True
7    False
8    False
dtype: object

In [67]:
s4.str.contains('A', na=False)

0     True
1    False
2    False
3     True
4    False
5    False
6     True
7    False
8    False
dtype: bool

In [68]:
s4.str.contains('A', na=True)

0     True
1    False
2    False
3     True
4    False
5     True
6     True
7    False
8    False
dtype: bool

## Creating Indicator Variables

You can extract dummy variables from string columns. For example if they are separated by a '|':

In [69]:
s = pd.Series(['a', 'a|b', np.nan, 'a|c'])

In [70]:
s

0      a
1    a|b
2    NaN
3    a|c
dtype: object

In [71]:
s.str.get_dummies(sep='|')

,a,b,c
0,1,0,0
1,1,1,0
2,0,0,0
3,1,0,1


String Index also supports get_dummies which returns a MultiIndex.

New in version 0.18.1.

In [72]:
idx = pd.Index(['a', 'a|b', np.nan, 'a|c'])

In [73]:
idx

Index([u'a', u'a|b', nan, u'a|c'], dtype='object')

In [74]:
idx.str.get_dummies(sep='|')

MultiIndex(levels=[[0, 1], [0, 1], [0, 1]],
           labels=[[1, 1, 0, 1], [0, 1, 0, 0], [0, 0, 0, 1]],
           names=[u'a', u'b', u'c'])

See also **[get_dummies()](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html#pandas.get_dummies).**

## Method Summary

<table border="1" class="docutils" id="text-summary" align="left">
<colgroup>
<col width="20%" />
<col width="80%" />
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Method</th>
<th class="head">Description</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.cat.html#pandas.Series.str.cat" title="pandas.Series.str.cat"><tt class="xref py py-meth docutils literal"><span class="pre">cat()</span></tt></a></td>
<td>Concatenate strings</td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.split.html#pandas.Series.str.split" title="pandas.Series.str.split"><tt class="xref py py-meth docutils literal"><span class="pre">split()</span></tt></a></td>
<td>Split strings on delimiter</td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.rsplit.html#pandas.Series.str.rsplit" title="pandas.Series.str.rsplit"><tt class="xref py py-meth docutils literal"><span class="pre">rsplit()</span></tt></a></td>
<td>Split strings on delimiter working from the end of the string</td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.get.html#pandas.Series.str.get" title="pandas.Series.str.get"><tt class="xref py py-meth docutils literal"><span class="pre">get()</span></tt></a></td>
<td>Index into each element (retrieve i-th element)</td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.join.html#pandas.Series.str.join" title="pandas.Series.str.join"><tt class="xref py py-meth docutils literal"><span class="pre">join()</span></tt></a></td>
<td>Join strings in each element of the Series with passed separator</td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.get_dummies.html#pandas.Series.str.get_dummies" title="pandas.Series.str.get_dummies"><tt class="xref py py-meth docutils literal"><span class="pre">get_dummies()</span></tt></a></td>
<td>Split strings on the delimiter returning DataFrame of dummy variables</td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.contains.html#pandas.Series.str.contains" title="pandas.Series.str.contains"><tt class="xref py py-meth docutils literal"><span class="pre">contains()</span></tt></a></td>
<td>Return boolean array if each string contains pattern/regex</td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.replace.html#pandas.Series.str.replace" title="pandas.Series.str.replace"><tt class="xref py py-meth docutils literal"><span class="pre">replace()</span></tt></a></td>
<td>Replace occurrences of pattern/regex with some other string</td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.repeat.html#pandas.Series.str.repeat" title="pandas.Series.str.repeat"><tt class="xref py py-meth docutils literal"><span class="pre">repeat()</span></tt></a></td>
<td>Duplicate values (<tt class="docutils literal"><span class="pre">s.str.repeat(3)</span></tt> equivalent to <tt class="docutils literal"><span class="pre">x</span> <span class="pre">*</span> <span class="pre">3</span></tt>)</td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.pad.html#pandas.Series.str.pad" title="pandas.Series.str.pad"><tt class="xref py py-meth docutils literal"><span class="pre">pad()</span></tt></a></td>
<td>Add whitespace to left, right, or both sides of strings</td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.center.html#pandas.Series.str.center" title="pandas.Series.str.center"><tt class="xref py py-meth docutils literal"><span class="pre">center()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.center</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.ljust.html#pandas.Series.str.ljust" title="pandas.Series.str.ljust"><tt class="xref py py-meth docutils literal"><span class="pre">ljust()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.ljust</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.rjust.html#pandas.Series.str.rjust" title="pandas.Series.str.rjust"><tt class="xref py py-meth docutils literal"><span class="pre">rjust()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.rjust</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.zfill.html#pandas.Series.str.zfill" title="pandas.Series.str.zfill"><tt class="xref py py-meth docutils literal"><span class="pre">zfill()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.zfill</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.wrap.html#pandas.Series.str.wrap" title="pandas.Series.str.wrap"><tt class="xref py py-meth docutils literal"><span class="pre">wrap()</span></tt></a></td>
<td>Split long strings into lines with length less than a given width</td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.slice.html#pandas.Series.str.slice" title="pandas.Series.str.slice"><tt class="xref py py-meth docutils literal"><span class="pre">slice()</span></tt></a></td>
<td>Slice each string in the Series</td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.slice_replace.html#pandas.Series.str.slice_replace" title="pandas.Series.str.slice_replace"><tt class="xref py py-meth docutils literal"><span class="pre">slice_replace()</span></tt></a></td>
<td>Replace slice in each string with passed value</td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.count.html#pandas.Series.str.count" title="pandas.Series.str.count"><tt class="xref py py-meth docutils literal"><span class="pre">count()</span></tt></a></td>
<td>Count occurrences of pattern</td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.startswith.html#pandas.Series.str.startswith" title="pandas.Series.str.startswith"><tt class="xref py py-meth docutils literal"><span class="pre">startswith()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.startswith(pat)</span></tt> for each element</td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.endswith.html#pandas.Series.str.endswith" title="pandas.Series.str.endswith"><tt class="xref py py-meth docutils literal"><span class="pre">endswith()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.endswith(pat)</span></tt> for each element</td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.findall.html#pandas.Series.str.findall" title="pandas.Series.str.findall"><tt class="xref py py-meth docutils literal"><span class="pre">findall()</span></tt></a></td>
<td>Compute list of all occurrences of pattern/regex for each string</td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.match.html#pandas.Series.str.match" title="pandas.Series.str.match"><tt class="xref py py-meth docutils literal"><span class="pre">match()</span></tt></a></td>
<td>Call <tt class="docutils literal"><span class="pre">re.match</span></tt> on each element, returning matched groups as list</td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.extract.html#pandas.Series.str.extract" title="pandas.Series.str.extract"><tt class="xref py py-meth docutils literal"><span class="pre">extract()</span></tt></a></td>
<td>Call <tt class="docutils literal"><span class="pre">re.search</span></tt> on each element, returning DataFrame with one row for each element and one column for each regex capture group</td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.extractall.html#pandas.Series.str.extractall" title="pandas.Series.str.extractall"><tt class="xref py py-meth docutils literal"><span class="pre">extractall()</span></tt></a></td>
<td>Call <tt class="docutils literal"><span class="pre">re.findall</span></tt> on each element, returning DataFrame with one row for each match and one column for each regex capture group</td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.len.html#pandas.Series.str.len" title="pandas.Series.str.len"><tt class="xref py py-meth docutils literal"><span class="pre">len()</span></tt></a></td>
<td>Compute string lengths</td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.strip.html#pandas.Series.str.strip" title="pandas.Series.str.strip"><tt class="xref py py-meth docutils literal"><span class="pre">strip()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.strip</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.rstrip.html#pandas.Series.str.rstrip" title="pandas.Series.str.rstrip"><tt class="xref py py-meth docutils literal"><span class="pre">rstrip()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.rstrip</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.lstrip.html#pandas.Series.str.lstrip" title="pandas.Series.str.lstrip"><tt class="xref py py-meth docutils literal"><span class="pre">lstrip()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.lstrip</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.partition.html#pandas.Series.str.partition" title="pandas.Series.str.partition"><tt class="xref py py-meth docutils literal"><span class="pre">partition()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.partition</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.rpartition.html#pandas.Series.str.rpartition" title="pandas.Series.str.rpartition"><tt class="xref py py-meth docutils literal"><span class="pre">rpartition()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.rpartition</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.lower.html#pandas.Series.str.lower" title="pandas.Series.str.lower"><tt class="xref py py-meth docutils literal"><span class="pre">lower()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.lower</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.upper.html#pandas.Series.str.upper" title="pandas.Series.str.upper"><tt class="xref py py-meth docutils literal"><span class="pre">upper()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.upper</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.find.html#pandas.Series.str.find" title="pandas.Series.str.find"><tt class="xref py py-meth docutils literal"><span class="pre">find()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.find</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.rfind.html#pandas.Series.str.rfind" title="pandas.Series.str.rfind"><tt class="xref py py-meth docutils literal"><span class="pre">rfind()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.rfind</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.index.html#pandas.Series.str.index" title="pandas.Series.str.index"><tt class="xref py py-meth docutils literal"><span class="pre">index()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.index</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.rindex.html#pandas.Series.str.rindex" title="pandas.Series.str.rindex"><tt class="xref py py-meth docutils literal"><span class="pre">rindex()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.rindex</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.capitalize.html#pandas.Series.str.capitalize" title="pandas.Series.str.capitalize"><tt class="xref py py-meth docutils literal"><span class="pre">capitalize()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.capitalize</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.swapcase.html#pandas.Series.str.swapcase" title="pandas.Series.str.swapcase"><tt class="xref py py-meth docutils literal"><span class="pre">swapcase()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.swapcase</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.normalize.html#pandas.Series.str.normalize" title="pandas.Series.str.normalize"><tt class="xref py py-meth docutils literal"><span class="pre">normalize()</span></tt></a></td>
<td>Return Unicode normal form. Equivalent to <tt class="docutils literal"><span class="pre">unicodedata.normalize</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.translate.html#pandas.Series.str.translate" title="pandas.Series.str.translate"><tt class="xref py py-meth docutils literal"><span class="pre">translate()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.translate</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.isalnum.html#pandas.Series.str.isalnum" title="pandas.Series.str.isalnum"><tt class="xref py py-meth docutils literal"><span class="pre">isalnum()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.isalnum</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.isalpha.html#pandas.Series.str.isalpha" title="pandas.Series.str.isalpha"><tt class="xref py py-meth docutils literal"><span class="pre">isalpha()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.isalpha</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.isdigit.html#pandas.Series.str.isdigit" title="pandas.Series.str.isdigit"><tt class="xref py py-meth docutils literal"><span class="pre">isdigit()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.isdigit</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.isspace.html#pandas.Series.str.isspace" title="pandas.Series.str.isspace"><tt class="xref py py-meth docutils literal"><span class="pre">isspace()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.isspace</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.islower.html#pandas.Series.str.islower" title="pandas.Series.str.islower"><tt class="xref py py-meth docutils literal"><span class="pre">islower()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.islower</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.isupper.html#pandas.Series.str.isupper" title="pandas.Series.str.isupper"><tt class="xref py py-meth docutils literal"><span class="pre">isupper()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.isupper</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.istitle.html#pandas.Series.str.istitle" title="pandas.Series.str.istitle"><tt class="xref py py-meth docutils literal"><span class="pre">istitle()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.istitle</span></tt></td>
</tr>
<tr class="row-odd"><td><a class="reference internal" href="generated/pandas.Series.str.isnumeric.html#pandas.Series.str.isnumeric" title="pandas.Series.str.isnumeric"><tt class="xref py py-meth docutils literal"><span class="pre">isnumeric()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.isnumeric</span></tt></td>
</tr>
<tr class="row-even"><td><a class="reference internal" href="generated/pandas.Series.str.isdecimal.html#pandas.Series.str.isdecimal" title="pandas.Series.str.isdecimal"><tt class="xref py py-meth docutils literal"><span class="pre">isdecimal()</span></tt></a></td>
<td>Equivalent to <tt class="docutils literal"><span class="pre">str.isdecimal</span></tt></td>
</tr>
</tbody>
</table>